In [14]:

import sys
import os
import torch
import pandas as pd
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc
import json
from torchvision import transforms
import matplotlib.pyplot as plt
import glob
from PIL import Image


In [15]:
import importlib
import utils
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [17]:
positive_path = glob.glob(
    # "C:/Users/yuuta/Documents/fashion/data/negative_coordinates/**.json"
    "C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
    # negative_coordinates_random
)

In [18]:
targets = []
p_cnt = 0
for fp in positive_path:
    json_dict = open_json(fp)
    items = filter_basic_items(json_dict["items"])
    attributes = []
    vectors = []
    for item in items:
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        targets.append(itemId)

In [19]:
from model_learning.model_structure.image_encoder import ImageEncoderV2


model = ImageEncoderV2().to(device)

model.load_state_dict(
    torch.load(
        "C:/Users/yuuta/Documents/fashion/model_learning/compatibility/models/triplet-image-2024-01-09.pth"
    )
)
def infer(input_img):
    with torch.no_grad():
        pred = model(input_img)
    return pred

c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


def to_image(itemId):
    image_path = f"C:/Users/yuuta/Documents/fashion/data/images/{itemId}.jpg"
    image = Image.open(image_path)
    if image.mode != "RGB":
        image = image.convert("L")
        image = Image.merge("RGB", [image] * 3)

    image = transform(image)
    return image


ids = []
# captions = []
images = []
vectors = {}
BATCH_SIZE = 32
cnt = 0


In [32]:

for i in range(0, len(targets), BATCH_SIZE):
    if i % 100 == 0:
        print("{i * 100 / len(targets)} %", end='')
    s = i
    e = min(i + BATCH_SIZE, len(targets))

    images = torch.stack(list(map(to_image, targets[s:e]))).to(torch.device("cuda"))
    pred = infer(images)
    pred_ids = targets[s:e]
    for id, vector in zip(targets[s:e], pred):
        vectors[id] = vector.to("cpu").tolist()


0.0 %0.6838132847825901 %1.3676265695651801 %2.0514398543477705 %

In [26]:

with open("C:/Users/yuuta/Documents/fashion/model_learning/compatibility/data/id_to_vector.json", "w") as f:
    json.dump(vectors, f)

TypeError: Object of type Tensor is not JSON serializable

In [30]:
list(vectors.values())[:5]

[tensor([-10.7023, -12.5416, -17.5678,  -6.4852,   6.3120,  15.4253,   6.4901,
          18.5119,  -9.4319,  37.4372, -18.5005,  19.6943, -20.1225,  17.4573,
         -18.2823,  -1.4168, -23.8998, -12.5234, -23.0396, -19.0081,  -2.4001,
           8.8445,  36.8185,   2.5115, -13.4946,  19.3297,  16.2160, -38.0593,
           1.2427, -12.4501,   2.5666,  11.4934,  10.1989,  11.5499,  -5.5777,
         -11.0189,  19.1889, -28.0704,   6.8150, -31.7473,   9.6765, -19.9543,
          21.8306, -34.8923, -19.3117, -21.4430,  23.8191,  -7.1323,  38.2197,
         -16.1099, -16.8643,   9.9377,  -1.5241,  -9.7206,   0.4001,  28.0984,
         -36.6874,  10.8236,  42.3823, -18.4551, -18.4919,  -2.7195,  17.0498,
          -5.8643], device='cuda:0'),
 tensor([-42.9708,  33.2625,  -5.2875,  48.3588,   1.5704,  30.3841,  32.8135,
         -32.9735,  37.3370,  22.0613, -12.7042, -12.9986, -27.4082,  41.2355,
           1.9496,   4.5776, -43.3037,  -4.0376, -35.3809, -16.3294, -34.8197,
          -5.6